In [1]:
import os
import tensorflow as tf
from PIL import Image
from nets import nets_factory
import numpy as np

In [2]:
# 不同字符数量
CHAR_SET_LEN = 10
# 图片高度
IMAGE_HEIGHT = 60
# 图片宽度
IMAGE_WIDTH = 160
# 批次
BATCH_SIZE = 5
# tfrecord文件存放路径
TFRECORD_FILE = "/home/hjy/Myproject/tensorflow/captcha/test.tfrecords"

# placeholder
x = tf.placeholder(tf.float32, [None, 224, 224])

# 从tfrecord独处数据
def read_and_decode(filename):
    # 根据文件生成一个队列
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    # 返回文件名和文件
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example, 
                                     features={
                                         'image': tf.FixedLenFeature([], tf.string),
                                         'label0': tf.FixedLenFeature([], tf.int64),
                                         'label1': tf.FixedLenFeature([], tf.int64),
                                         'label2': tf.FixedLenFeature([], tf.int64),
                                         'label3': tf.FixedLenFeature([], tf.int64),
                                     })
    # 获取图片数据
    image = tf.decode_raw(features['image'], tf.uint8)
    # 没有预处理的灰度图
    image_raw = tf.reshape(image, [224, 224])
    # tf.train.shuffle_batch必须确定shape
    image = tf.reshape(image, [224, 224])
    # 图片预处理
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    # 获取label
    label0 = tf.cast(features['label0'], tf.int32)
    label1 = tf.cast(features['label1'], tf.int32)
    label2 = tf.cast(features['label2'], tf.int32)
    label3 = tf.cast(features['label3'], tf.int32)
    return image,image_raw, label0, label1, label2, label3


In [6]:
# 获取图片数据和标签
image, image_raw, label0, label1, label2, label3 = read_and_decode(TFRECORD_FILE)
# 使用shuffle_batch可以随机打乱
image_batch, image_raw_batch, label_batch0, label_batch1, label_batch2, label_batch3 = tf.train.shuffle_batch(
        [image, image_raw, label0, label1, label2, label3], batch_size = BATCH_SIZE, 
        capacity = 50000, min_after_dequeue=10000, num_threads=1) 
# 定义网络结构read_and_decode
train_network_fn = nets_factory.get_network_fn(
    'alexnet_v2',
    num_classes=CHAR_SET_LEN,
    weight_decay=0.0005,
    is_training=False)

with tf.Session() as sess:
    # inputs: a tensor of size [batch_size, height, width, channels]
    X = tf.reshape(x, [BATCH_SIZE, 224, 224, 1])
    # 数据输入网络得到输出值
    logits0, logits1, logits2, logits3, end_points = train_network_fn(X)
    
   
    # 计算预测值
    predict0 = tf.reshape(logits0, [-1, CHAR_SET_LEN])
    predict0 = tf.arg_max(predict0, 1)
    
    predict1 = tf.reshape(logits1, [-1, CHAR_SET_LEN])
    predict1 = tf.arg_max(predict1, 1)
    
    predict2 = tf.reshape(logits2, [-1, CHAR_SET_LEN])
    predict2 = tf.arg_max(predict2, 1)
    
    predict3 = tf.reshape(logits3, [-1, CHAR_SET_LEN])
    predict3 = tf.arg_max(predict3, 1)
    

    # 初始化
    sess.run(tf.global_variables_initializer())
    # 载入训练好的模型
    saver = tf.train.Saver()
    saver.restore(sess, './captcha/models/crack_captcha.model-6000')
    # 创建一个协调器，管理线程
    coord = tf.train.Coordinator()
    # 启动QueueRunner,此时文件名已经进队
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    for i in range(10):
        # 获取一个批次的数据和标签
        b_image, b_image_raw, b_label0, b_label1, b_label2, b_label3 = sess.run([image_batch, image_raw_batch, label_batch0, label_batch1, label_batch2, label_batch3])
        # 显示图片
        img = image.fromarray(b_image_raw[0], 'L')
        plt.imshow(img)
        plt.show()
        # 打印标签
        print('label:', b_label0, b_label1, b_label2, b_label3)
        # 预测
        label0, label1, label2, label3 = sess.run([predict0, predict1, predict2, predict3], feed_dict={x: b_image})
        # 打印预测值
        print('predict:', label0, label1, label2, label3)
    # 通知其他线程关闭
    coord.request_stop()

    # 其他所有线程关闭之后，这一函数才能返回
    coord.join(threads)

Instructions for updating:
Use `argmax` instead
INFO:tensorflow:Restoring parameters from ./captcha/models/crack_captcha.model-6000


InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on ./captcha/models/crack_captcha.model-6000: Not found: ./captcha/models; No such file or directory
	 [[Node: save/RestoreV2_23 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2_23/tensor_names, save/RestoreV2_23/shape_and_slices)]]

Caused by op 'save/RestoreV2_23', defined at:
  File "/home/hjy/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hjy/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hjy/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hjy/anaconda3/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File